In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
from sklearn import decomposition
from sklearn import preprocessing
from sklearn import manifold
from sklearn import metrics
from tqdm.auto import tqdm
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from skopt import BayesSearchCV
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, mean_squared_error, explained_variance_score

import warnings
warnings.filterwarnings('ignore')

# For reproducibility
np.random.seed(42)

In [2]:
def dataset_split(X, Y, ratio):
    """ Function to split the dataset into train and test """
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=ratio)
    return X_train, X_test, y_train, y_test

def score(y_true,y_pred): 
    """ Function to print the metrics of interest of the model """
    mse = mean_squared_error(y_true, y_pred) #set score here and not below if using MSE in GridCV
    r2 = r2_score(y_true, y_pred)
    ev = explained_variance_score(y_true, y_pred)
    print("MSE is: ", mse)
    print("R2 is: ", r2)
    print("Explained variance is:", ev)
    
def model_tune(model_name, params, X_train, y_train):
    if model_name == 'knn':
        model = KNeighborsRegressor(algorithm='auto')
    elif model_name == 'rf':
        model = RandomForestRegressor()
    elif model_name == 'regression':
        model = ElasticNet()
    else:
        print('Model unrecognised')
    # Tune the model with Bayesian optimisation
    opt = BayesSearchCV(model, param_grid, n_iter=30, cv=5, verbose=1)
    opt.fit(X_train, y_train)
    # With the following parameter combination being optimal
    print("Best parameter combo:", opt.best_params_)
    # Having the following score
    print("Best validation MSE:", opt.best_score_)
    return opt.best_estimator_

# Load the data

In [6]:
# Path to dataset
PATH = '/cdtshared/wearables/students/group5/'

# Features from biobank
X_train = pd.read_pickle(PATH+"XtrainPCA.pkl")
X_val = pd.read_pickle(PATH+"XvalPCA.pkl")
X_test = pd.read_pickle(PATH+"XtestPCA.pkl")
# Outcome
y_train = pd.read_pickle(PATH+"ytrainPCA.pkl")
y_val = pd.read_pickle(PATH+"yvalPCA.pkl")
y_test = pd.read_pickle(PATH+"ytestPCA.pkl")

# Model Tuning

In [17]:
# Define the hyperparameters you want to sweep through (important it is manual for generalisation)
# C for regularisation if doing regression
# kernel if doing SVM for example

# In this case we are tuning for ElasticNet hyperparameters
# Regularisation constant
alpha = [0.0001, 0.001, 0.01, 0.1, 1, 10, 100]
# Relative ratio of l1 vs l2 regularisation
l1_ratio = np.arange(0.0, 1.0, 0.1)

# Create the grid
param_grid = {'alpha': alpha,
               'l1_ratio': l1_ratio}

In [18]:
# Define the model you are interested in
model = ElasticNet()

In [19]:
# Create the fold corresponding to our own train and validation split
X = np.vstack((X_train, X_val))
test_fold = [-1 for _ in range(X_train.shape[0])] + [0 for _ in range(X_val.shape[0])]
y = np.concatenate([y_train, y_val])
ps = PredefinedSplit(test_fold)

In [20]:
# Skip to bayesian below if taking too long to compute
clf = GridSearchCV(model, param_grid, cv=ps, refit=False)
clf.fit(X, y)

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             estimator=ElasticNet(),
             param_grid={'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                         'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])},
             refit=False)

In [22]:
# With the following parameter combination being optimal
print("Best parameter combo:", clf.best_params_)
# Having the following score
print("Best validation MSE:", clf.best_score_)

Best parameter combo: {'alpha': 10, 'l1_ratio': 0.2}
Best validation MSE: 0.026983623287271108


In [10]:
# Try with Bayesian optimisation for faster computation of tuning
opt = BayesSearchCV(model, param_grid, n_iter=30, cv=ps, verbose=1, refit=False)
opt.fit(X, y)

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fi

BayesSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
              estimator=ElasticNet(), n_iter=30, refit=False,
              search_spaces={'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
                             'l1_ratio': array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])},
              verbose=1)

In [11]:
# Get the best model
#model_best = opt.best_estimator_
# With the following parameter combination being optimal
print("Best parameter combo:", opt.best_params_)
# Having the following score
print("Best validation MSE:", opt.best_score_)

Best parameter combo: OrderedDict([('alpha', 10.0), ('l1_ratio', 0.2)])
Best validation MSE: 2.6983623287271108e-02


Best parameters: <br>
**alpha**:10 <br>
**l1_ratio**: 0.2 <br>

In [14]:
model = ElasticNet(alpha=10, l1_ratio=0.2)
model.fit(X_train, y_train)

ElasticNet(alpha=10, l1_ratio=0.2)

In [15]:
score(y_val, model.predict(X_val))

MSE is:  87.83612820485148
R2 is:  0.026983623287271108
Explained variance is: 0.027005833933342438


In [16]:
# Get the test set performance
score(y_test, model.predict(X_test))

MSE is:  84.6287633062459
R2 is:  0.03593913359722245
Explained variance is: 0.03597720096181878
